# [OSMsc](https://github.com/ruirzma/osmsc) spatial semantics.

The main content of this notebook is spatial semantics.

In [1]:
import osmsc as oc
import osmnx as ox
import numpy as np

OSMsc has been successfully imported!!!


In [2]:
cityName = "Paris"
bbox_Paris = (48.8695, 2.3908, 48.8825, 2.4108)

In [3]:
BG = oc.building_group(bbox= bbox_Paris)
building_gdf = BG.get_gdf(tags=True, building_levels=True)
building_gdf_prj = ox.project_gdf(building_gdf)

# If the external building height dataset is unavailable,
# Assume building level is 3m
building_gdf["building_levels"] = building_gdf["building_levels"].replace(np.nan, 0)
building_gdf["Building_height"] = building_gdf["building_levels"] * 3
building_gdf.head()

,osmid,geometry,tags,building_levels,osmscID,Building_area,Building_perimeter,Building_height
0,63212010,"POLYGON ((2.39680 48.88232, 2.39673 48.88237, ...","{'building': 'yes', 'building:levels': '9', 's...",9,Building_63212010,425.941590,126.101343,27
1,63212012,"POLYGON ((2.39486 48.87986, 2.39472 48.87985, ...","{'building': 'house', 'source': 'cadastre-dgi-...",1,Building_63212012,42.765888,29.051447,3
2,63212018,"POLYGON ((2.39156 48.88066, 2.39154 48.88068, ...","{'building': 'yes', 'building:levels': '1', 's...",1,Building_63212018,58.690170,33.174859,3
3,63212023,"POLYGON ((2.39126 48.88133, 2.39127 48.88134, ...","{'building': 'yes', 'source': 'cadastre-dgi-fr...",1,Building_63212023,81.502088,43.837655,3
4,63212025,"POLYGON ((2.39941 48.87697, 2.39939 48.87696, ...","{'building': 'yes', 'building:levels': '4', 's...",4,Building_63212025,47.378189,27.752066,12


In [4]:
VG = oc.vegetation_group(bbox = bbox_Paris)
VG_gdf = VG.get_gdf()
VG_gdf_prj = ox.project_gdf(VG_gdf)

In [5]:
WG = oc.waterbody_group(bbox =bbox_Paris)
WG_gdf = WG.get_gdf()
WG_gdf_prj = ox.project_gdf(WG_gdf)

In [6]:
UG = oc.urban_tile_group(bbox= bbox_Paris, trans_type= ['primary','secondary', 'tertiary'
                                                                   ,'residential','motorway','motorway_link'])
urban_tile_gdf_prj = UG.get_gdf_prj()

In [7]:
urban_tile_gdf_prj = oc.feature.add_spatial_semantics_attr(urban_tile_gdf_prj,building_gdf_prj,"containsBuilding")
urban_tile_gdf_prj = oc.feature.add_spatial_semantics_attr(urban_tile_gdf_prj,VG_gdf_prj,"containsVegetation")
urban_tile_gdf_prj = oc.feature.add_spatial_semantics_attr(urban_tile_gdf_prj,WG_gdf_prj,"containsWaterbody")

In [8]:
building_gdf_prj = oc.feature.add_spatial_semantics_attr(building_gdf_prj,urban_tile_gdf_prj,"withinUrbanTile")
VG_gdf_prj = oc.feature.add_spatial_semantics_attr(VG_gdf_prj,urban_tile_gdf_prj,"withinUrbanTile")
WG_gdf_prj = oc.feature.add_spatial_semantics_attr(WG_gdf_prj,urban_tile_gdf_prj,"withinUrbanTile")

In [9]:
building_gdf_prj.head(2)

,osmid,geometry,tags,building_levels,osmscID,Building_area,Building_perimeter,Building_height,withinUrbanTile
0,63212010,"POLYGON ((455776.771 5414549.319, 455771.794 5...","{'building': 'yes', 'building:levels': '9', 's...",9,Building_63212010,425.941590,126.101343,27,[UrbanTile_101]
1,63212012,"POLYGON ((455632.293 5414277.024, 455621.889 5...","{'building': 'house', 'source': 'cadastre-dgi-...",1,Building_63212012,42.765888,29.051447,3,[UrbanTile_64]


In [10]:
VG_gdf_prj.head(2)

,osmid,geometry,tags,osmscID,Vegetation_area,Vegetation_perimeter,withinUrbanTile
0,19578842,"POLYGON ((455790.389 5414633.833, 455760.527 5...","{'barrier': 'fence', 'internet_access': 'wlan'...",Vegetation_19578842,46457.250675,891.458691,[UrbanTile_101]
1,19579673,"POLYGON ((455981.772 5413648.562, 455983.907 5...","{'barrier': 'wall', 'landuse': 'cemetery', 'na...",Vegetation_19579673,16219.725849,588.740451,[UrbanTile_50]


In [11]:
WG_gdf_prj.head(2)

,osmid,geometry,tags,osmscID,Waterbody_area,Waterbody_perimeter,withinUrbanTile
0,29280785,"POLYGON ((456348.686 5413309.948, 456455.663 5...","{'access': 'no', 'architect': 'Eugène Belgrand...",WaterBody_29280785,22047.053978,571.839387,[UrbanTile_26]
1,63237122,"POLYGON ((456426.277 5414108.045, 456461.832 5...","{'access': 'no', 'architect': 'Eugène Belgrand...",WaterBody_63237122,18289.473141,542.477899,[UrbanTile_69]


In [12]:
urban_tile_gdf_prj.head(2)

,geometry,osmscID,UrbanTile_area,UrbanTile_perimeter,containsBuilding,containsVegetation,containsWaterbody
0,"POLYGON ((455319.368 5413938.058, 455319.574 5...",UrbanTile_0,14206.078301,550.042857,"[Building_63232641, Building_63233862, Buildin...","[Vegetation_1076825084, Vegetation_536464856]",[nan]
1,"POLYGON ((455464.009 5413939.332, 455464.775 5...",UrbanTile_1,28787.141232,654.760361,"[Building_63241205, Building_63237978, Buildin...","[Vegetation_368974898, Vegetation_542011324, V...",[nan]
